In [1]:
#中時新聞網
#導入所需套件
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time as t
import datetime
import os
from urllib.parse import urljoin

# 輸入要搜尋的五檔股票
search_lis =[]
x1 ='長榮'
search_lis.append(x1)
x2 ='陽明'
search_lis.append(x2)
x3 ='萬海'
search_lis.append(x3)
x4 ='群創' 
search_lis.append(x4)
x5 ='友達' 
search_lis.append(x5)

# 先定義能判別日期的函數(只抓取近兩天的新聞)
def distinguish_date(time):    
    time = time.replace('/','-')
    yest = datetime.datetime.now()+ datetime.timedelta(days=-1)
    yest  = yest .strftime("%Y-%m-%d")
    if ( time == datetime.datetime.now().strftime("%Y-%m-%d") ) or ( time == yest  ):
        return True
    else:
        return False

# 開始爬蟲的時間，為了區別每次爬蟲
start_time = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")

# 每檔股票都跑一次
for stock in search_lis:
    title_lis = []
    time_lis = []
    source_lis = []
    author_lis = []
    article_url_lis = []
    
    # 跑每頁的內容
    page = 1
    while 1:    
        url = "https://wantrich.chinatimes.com/"
        url = url + 'search/' + stock + '/' + str(page)
        re = requests.get(url)
        soup = BeautifulSoup(re.text, "html.parser")
        news = soup.find('ul',{'class':'vertical-list'})
        news = news.findAll('div', {'class':'articlebox-compact'})

        for new in news:  
            new_time = new.find('div', {'class':'meta-info'})
            time = new_time.find('span', {'class':'date'}).text

            if distinguish_date(time) == True :
                new_title = new.find("h3",{'class':'title'})
                title = new_title.text       

                article_url = new.find('a')['href']
                article_url = urljoin(url,article_url )
                
                #隔頁爬蟲
                re = requests.get(article_url)
                soup = BeautifulSoup(re.text, "html.parser")
                soup = soup.find('div', {'class':'align-items-center'})
                source = soup.find('div',{'class':'source'}).text
                author = soup.find('div',{'class':'author'}).text
                if author == '':
                    author = '(無特定作者)'
                # 確切新聞發布時間
                time = time + '   ' + soup.find('span',{'class':'hour'}).text  

                #
                title_lis.append(title)
                time_lis.append(time)
                source_lis.append(source)
                author_lis.append(author)
                article_url_lis.append(article_url)
                t.sleep(1)
            elif distinguish_date(time) == False :
                break
    
        if distinguish_date(time) == False :
            break
        page = page + 1
        
    df = pd.DataFrame({"title":title_lis,"time": time_lis ,"author":author_lis ,"link":article_url_lis})
    # 設定檔案儲存路徑，並輸出成excel檔
    path = "D:\python documents"
    path = os.path.join(path,'中時日報',start_time,stock)    
    #print(stock)
    #print(path)
    if not os.path.exists(path):
        os.makedirs(path)
    string = stock + '_News_chinatimes.xlsx'
    df.to_excel(os.path.join(path,string))

        

KeyboardInterrupt: 

In [2]:
#經濟日報
#導入所需套件
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time as t
import datetime
import os
from urllib.parse import urljoin

# 輸入要搜尋的五檔股票
search_lis =[]
x1 ='長榮'
search_lis.append(x1)
x2 ='陽明'
search_lis.append(x2)
x3 ='萬海'
search_lis.append(x3)
x4 ='群創' 
search_lis.append(x4)
x5 ='友達' 
search_lis.append(x5)

# 先定義能判別日期的函數(只抓取近兩天的新聞)
def distinguish_date(time):    
    time = time.replace('/','-')
    yest = datetime.datetime.now()+ datetime.timedelta(days=-1)
    yest  = yest .strftime("%Y-%m-%d")
    if ( time == datetime.datetime.now().strftime("%Y-%m-%d") ) or ( time == yest  ):
        return True
    else:
        return False

# 開始爬蟲的時間，為了區別每次爬蟲
start_time = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")

# 每檔股票都跑一次
for stock in search_lis:
    title_lis = []
    time_lis = []
    author_lis = []
    article_url_lis = []
    
    # 開始爬蟲
    url = "https://money.udn.com/search/result/1001/"
    url = url + stock
    re = requests.get(url)
    soup = BeautifulSoup(re.text, "html.parser")
    news = soup.find('ul',{'class':'story-list-holder'})
    news = news.findAll('div', {'class':'story__content'})

    for new in news:  
        article_url = new.find('a')['href']
        # 隔頁爬蟲
        re = requests.get(article_url)
        soup = BeautifulSoup(re.text, "html.parser")
        time = soup.find('time',{'class':'article-body__time'}).text 
        # time 格式  '%Y/%m/%d %H:%M:%S' 
        time_spli = time.split(" ")
        if distinguish_date(time_spli[0]) == True :
            title = soup.find('h1',{'id':'story_art_title'}).text
            article__url = article_url
            
            # refine author 資訊 
            author = soup.find('div',{'class':'article-body__info'}).find('span').text
            author_spli = author.split("／")
            author_spli =  author_spli[0]
            author_spli = author_spli.split("/")
            author_spli =  author_spli[0].split(" ")
            #source = author_spli[0]
            author = author_spli[1]
            author = author.replace("記者",'')
            author = author.split("台北")
            author = author[0]
            if author == '':
                author = '(無特定作者)'
            
            #
            title_lis.append(title)
            time_lis.append(time)
            author_lis.append(author)
            article_url_lis.append(article__url)
            t.sleep(1)
        elif distinguish_date(time_spli[0]) == False :
            break
        
    df = pd.DataFrame({"title":title_lis,"time": time_lis ,"author":author_lis ,"link":article_url_lis})
    # 設定檔案儲存路徑，並輸出成excel檔
    path = "D:\python documents"
    path = os.path.join(path,'經濟日報',start_time,stock) 
    #print(stock)
    #print(path)
    if not os.path.exists(path):
        os.makedirs(path)
    string = stock + '_News_economy.xlsx'
    df.to_excel(os.path.join(path,string))

        

KeyboardInterrupt: 